In [2]:
import pandas as pd
import numpy as np
import os, sys
import glob

In [3]:
os.remove('./generation_forecast/.DS_Store')


FileNotFoundError: [Errno 2] No such file or directory: './generation_forecast/.DS_Store'

In [4]:
import glob

solaire_files = glob.glob('./generation_forecast/*Solaire*.csv')
eolien_files = glob.glob('./generation_forecast/*Eolien*.csv')
print(solaire_files)

['./generation_forecast/PrevisionSolaire_2020.xls.csv', './generation_forecast/PrevisionSolaire_2021.xls.csv', './generation_forecast/PrevisionSolaire_2017.xls.csv', './generation_forecast/PrevisionSolaire_2023.xls.csv', './generation_forecast/PrevisionSolaire_2019.xls.csv', './generation_forecast/PrevisionSolaire_2018.xls.csv', './generation_forecast/PrevisionSolaire_2022.xls.csv']


In [12]:
files = [solaire_files, eolien_files]

df_list_sol = []
for i, file in enumerate(solaire_files):

    df = pd.read_csv(file, encoding='latin1', sep='\t')
    df_list_sol.append(df)

df_list_eol = []
for i, file in enumerate(eolien_files):

    df = pd.read_csv(file, encoding='latin1', sep='\t')
    df_list_eol.append(df)


In [10]:
solaire_files

['./generation_forecast/PrevisionSolaire_2020.xls.csv',
 './generation_forecast/PrevisionSolaire_2021.xls.csv',
 './generation_forecast/PrevisionSolaire_2017.xls.csv',
 './generation_forecast/PrevisionSolaire_2023.xls.csv',
 './generation_forecast/PrevisionSolaire_2019.xls.csv',
 './generation_forecast/PrevisionSolaire_2018.xls.csv',
 './generation_forecast/PrevisionSolaire_2022.xls.csv']

In [42]:
dfteol = pd.concat(df_list_eol)
dfteol['Prévision J-1'].isna().sum()

0

In [43]:
dfteol

,Date,Heure,Prévision J-1,Taux de charge J-1,Prévision J H1,Prévision J H2,Prévision J H3,Prévision J H4,Prévision J H5,Prévision J H6,...,Taux de charge J H16,Taux de charge J H17,Taux de charge J H18,Taux de charge J H19,Taux de charge J H20,Taux de charge J H21,Taux de charge J H22,Taux de charge J H23,Taux de charge J H24,Unnamed: 52
0,01/01/2020,0:00,2966.13,19%,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,NaN
1,01/01/2020,1:00,2992.79,19%,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,NaN
2,01/01/2020,2:00,2857.19,18%,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,NaN
3,01/01/2020,3:00,2720.88,17%,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,NaN
4,01/01/2020,4:00,2584.39,16%,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8709,31/12/2018,19:00,1723.94,12%,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,NaN
8710,31/12/2018,20:00,1771.81,12%,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,NaN
8711,31/12/2018,21:00,1819.98,13%,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,NaN
8712,31/12/2018,22:00,1868.35,13%,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,NaN


In [13]:

df_solaire = pd.concat(df_list_sol)

#df_solaire = df_solaire.reset_index(inplace=False).drop('index', axis=1)

mask = df_solaire['Heure'].isna()
df_solaire=df_solaire.loc[~mask]

df_solaire['Heure'] = df_solaire['Heure'].apply(lambda x: "0" + x if len(x) == 4 else x)

df_solaire['Date_Heure_France'] = df_solaire['Date'] + ' ' + df_solaire['Heure']

df_solaire['Date_Heure_France'] = pd.to_datetime(df_solaire['Date_Heure_France'], format = "%d/%m/%Y %H:%M")

df_solaire = df_solaire.reset_index(drop=True)
df_solaire = df_solaire.sort_values(by='Date_Heure_France', axis = 0, ascending=True)

df_solaire = df_solaire[['Date_Heure_France', 'Prévision J-1']]
df_solaire.columns =['Date_Heure_France', 'Prévision J-1 Solaire'] 


date_debut = df_solaire['Date_Heure_France'].iloc[0]#.tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_fin = df_solaire['Date_Heure_France'].iloc[-1]#.tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_range = pd.date_range(date_debut, date_fin , freq='1H')
df_solaire = df_solaire.set_index('Date_Heure_France')
here = date_range.isin(df_solaire.index)
holes= date_range[~here]

for i, idx in enumerate(holes):

    df_solaire.loc[idx, 'Prévision J-1 Solaire'] = 0

df_solaire.loc[df_solaire.index.year == 2023]
date_debut = df_solaire.index[0].tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_range = pd.date_range(date_debut, periods = len(df_solaire), freq='1H')
df_solaire.index = date_range

df_solaire.index = df_solaire.index.rename('Date UTC')

df_solaire['Prévision J-1 Solaire'].str.startswith('-') == True

df_solaire.loc[df_solaire['Prévision J-1 Solaire'].str.startswith('-') == True] = 0
df_solaire['Prévision J-1 Solaire'] = df_solaire['Prévision J-1 Solaire'].astype(float)

df_solaire.sample(24)

,Prévision J-1 Solaire
Date UTC,
2019-11-07 03:00:00,0.00
2022-06-28 18:00:00,3476.51
2017-04-17 09:00:00,2465.16
2018-04-20 13:00:00,4144.07
2019-01-20 13:00:00,2573.29
2022-10-28 10:00:00,4923.38
2023-03-27 21:00:00,0.00
2019-05-27 22:00:00,0.00
2020-09-05 06:00:00,0.00


In [47]:

df_eolien = pd.concat(df_list_eol)

#df_eolien = df_eolien.reset_index(inplace=False).drop('index', axis=1)

#mask = df_eolien['Heure'].isna()
#df_eolien=df_eolien.loc[~mask]

df_eolien['Heure'] = df_eolien['Heure'].apply(lambda x: "0" + x if len(x) == 4 else x)

df_eolien['Date_Heure_France'] = df_eolien['Date'] + ' ' + df_eolien['Heure']

df_eolien['Date_Heure_France'] = pd.to_datetime(df_eolien['Date_Heure_France'], format = "%d/%m/%Y %H:%M")

df_eolien = df_eolien.reset_index(drop=True)
df_eolien = df_eolien.sort_values(by='Date_Heure_France', axis = 0, ascending=True)


df_eolien = df_eolien[['Date_Heure_France', 'Prévision J-1']]
df_eolien.columns = ['Date_Heure_France', 'Prévision J-1 Eolien']

date_debut = df_eolien['Date_Heure_France'].iloc[0]#.tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_fin = df_eolien['Date_Heure_France'].iloc[-1]#.tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_range = pd.date_range(date_debut, date_fin , freq='1H')
df_eolien = df_eolien.set_index('Date_Heure_France')
here = date_range.isin(df_eolien.index)
holes= date_range[~here]

for i, idx in enumerate(holes):

    df_eolien.loc[idx, 'Prévision J-1 Eolien'] = 0

df_eolien.loc[df_eolien.index.year == 2023]
date_debut = df_eolien.index[0].tz_localize('Europe/Paris').tz_convert('UTC').tz_localize(None)
date_range = pd.date_range(date_debut, periods = len(df_eolien), freq='1H')
df_eolien.index = date_range

df_eolien.index = df_eolien.index.rename('Date UTC')

df_eolien.sample(24)


,Prévision J-1 Eolien
Date UTC,
2021-10-06 22:00:00,0.00
2019-06-10 11:00:00,2617.37
2020-08-14 06:00:00,2668.84
2017-04-30 22:00:00,5850.00
2018-06-13 06:00:00,2007.84
2022-01-12 01:00:00,0.00
2017-05-27 07:00:00,2410.00
2020-08-29 18:00:00,784.90
2022-03-07 06:00:00,0.00


In [51]:
df_eolien

,Prévision J-1 Eolien
Date UTC,
2016-12-31 23:00:00,1051.0
2017-01-01 00:00:00,1060.0
2017-01-01 01:00:00,1071.0
2017-01-01 02:00:00,1081.0
2017-01-01 03:00:00,1228.0
...,...
2022-07-27 18:00:00,0.0
2022-07-27 19:00:00,0.0
2022-07-27 20:00:00,0.0


In [52]:
df_renew = df_solaire.join(df_eolien, how='inner')
df_renew.to_csv('/home/infres/dfitton-21/pfr/fil_rouge/examples/datasets/RENEW_FORECAST_FR.csv')

In [53]:
df_renew.isna().sum()

Prévision J-1 Solaire    0
Prévision J-1 Eolien     0
dtype: int64

In [54]:
df_renew

,Prévision J-1 Solaire,Prévision J-1 Eolien
Date UTC,,
2016-12-31 23:00:00,0.00,1051.0
2017-01-01 00:00:00,0.00,1060.0
2017-01-01 01:00:00,0.00,1071.0
2017-01-01 02:00:00,0.00,1081.0
2017-01-01 03:00:00,0.00,1228.0
...,...,...
2022-07-27 18:00:00,2614.66,0.0
2022-07-27 19:00:00,891.85,0.0
2022-07-27 20:00:00,30.93,0.0


In [63]:
init_data = pd.read_csv("../../../examples/datasets/EPEX_FR_NEW_UTC.csv",
                        index_col='Date')
init_data

,Load forecast,Generation forecast,Price
Date,,,
2017-01-01 00:00:00,73650.0,69539.0,58.23
2017-01-01 01:00:00,72350.0,67376.0,51.95
2017-01-01 02:00:00,68750.0,65412.0,47.27
2017-01-01 03:00:00,65900.0,64557.0,45.49
2017-01-01 04:00:00,65000.0,64552.0,44.50
...,...,...,...
2022-12-31 19:00:00,46950.0,44648.0,7.60
2022-12-31 20:00:00,44350.0,43841.0,3.69
2022-12-31 21:00:00,43950.0,44578.0,1.88


In [64]:
init_data.loc[init_data.isna().any(axis=1)]

,Load forecast,Generation forecast,Price
Date,,,
2019-02-17 23:00:00,59200.0,NaN,39.41
2019-02-18 00:00:00,57100.0,NaN,38.78
2019-02-18 01:00:00,57200.0,NaN,38.78
2019-02-18 02:00:00,54500.0,NaN,38.20
2019-02-18 03:00:00,53700.0,NaN,38.80
...,...,...,...
2021-05-28 17:00:00,46800.0,NaN,83.67
2021-05-28 18:00:00,45150.0,NaN,82.90
2021-05-28 19:00:00,43950.0,NaN,80.10


In [65]:
init_data.index = pd.to_datetime(init_data.index)

In [66]:
all_data = init_data.join(df_renew, how='inner')

# Pour les valeurs manquantes (Generation), on assigne la valeur Load
all_data['Generation forecast'].loc[all_data.isna().any(axis=1)==True] =all_data['Load forecast'].loc[all_data.isna().any(axis=1)==True]  

In [67]:
price = all_data.pop('Price')
all_data.insert(0,'Price' ,price)


In [68]:
all_data.to_csv('../../../examples/datasets/FR_NEW_UTC_W_RENEW.csv')

In [69]:
all_data['Prévision J-1 Eolien'].loc[all_data.isna().any(axis=1)]

Series([], Name: Prévision J-1 Eolien, dtype: float64)

In [70]:
all_data.loc[all_data['Prévision J-1 Eolien'] == 0]

,Price,Load forecast,Generation forecast,Prévision J-1 Solaire,Prévision J-1 Eolien
2021-04-20 10:00:00,78.27,56700.0,56132.0,2918.04,0.0
2021-04-20 11:00:00,72.10,56000.0,55557.0,3986.97,0.0
2021-04-20 12:00:00,68.89,52950.0,54589.0,4713.09,0.0
2021-04-20 13:00:00,66.57,50250.0,53399.0,4982.37,0.0
2021-04-20 14:00:00,66.01,49300.0,52531.0,5043.87,0.0
...,...,...,...,...,...
2022-07-27 18:00:00,650.00,47500.0,37589.0,2614.66,0.0
2022-07-27 19:00:00,645.00,45700.0,36742.0,891.85,0.0
2022-07-27 20:00:00,599.28,46000.0,37166.0,30.93,0.0
2022-07-27 21:00:00,500.47,47600.0,37846.0,0.00,0.0


In [71]:
all_data.loc[:,all_data.isna().any(axis=0)]

""
2017-01-01 00:00:00
2017-01-01 01:00:00
2017-01-01 02:00:00
2017-01-01 03:00:00
2017-01-01 04:00:00
...
2022-07-27 18:00:00
2022-07-27 19:00:00
2022-07-27 20:00:00
2022-07-27 21:00:00


In [72]:
all_data.index.isna().sum()

0

In [3]:
import pandas as pd
df = pd.read_csv('../../../examples/experimental_files/fc_nl_datFR_NEW_UTC_W_RENEW_YT2_CW728.csv')

In [12]:
df['Date'].loc[df.isna().any(axis=1)].values[0]

'2021-05-01'